In [1]:
'''
This is a global set of functions, to handle the diferent Mesh formats (2D-3D) obtained with GAMer.
''' 

import numpy as np 
import os
import bempp.api
import dolfin

def output_Path(x):
     return os.path.join('Molecule/',name, name + x)

def mesher(name , suffix=''):
    path = os.path.join('Molecule/',name, name + suffix)
    GAMer_Text = open(path).read().split('\n')   # Now the GAMer output is opened

    Vert_s  = GAMer_Text.index('vert=[ ') + 4 
    Vert_e  = GAMer_Text.index(']; ')-1  # Starting and ending point of the Vertex list

    Face_s = GAMer_Text.index('simp=[ ') + 4  # Starting point of the Faces list
    Face_e = len(GAMer_Text)-4

    Vertex_List = np.empty((0,3))
    for linea in GAMer_Text[Vert_s:Vert_e+1]:
        vert = map(float,linea.split()[-3:])
        Vertex_List = np.vstack( (Vertex_List,vert) )

    Vert_Text = open(output_Path('.vert'),'w+')
    for vert in Vertex_List:
        line = map(str,vert)
        Vert_Text.write( ' '.join(line) + '\n' )
    del Vertex_List
    Vert_Text.close()
    print('.vert File Ready')

    Faces_List = np.empty((0,4))
    for linea in GAMer_Text[Face_s:Face_e+1]:
        face = map(int,linea.split()[-4:])
        Faces_List = np.vstack( (Faces_List,face) )

    Face_Text = open(output_Path('.face'),'w+')
    for face in Faces_List:
        line = map(str,map(int,face) )
        Face_Text.write( ' '.join(line) + '\n' )
    del Faces_List
    Face_Text.close()
    print('.face File Ready')
    return None

def grid_factory(name , suffix='',output_format='.msh'):
    path = os.path.join('Molecule/',name, name + suffix)
    Vertex_Text = open( path+'.vert').read().split('\n')[:-1]

    factory = bempp.api.grid.GridFactory()

    for line in Vertex_Text:
        line = line.split()
        vert = map(float , line)
        factory.insert_vertex(vert)

    Faces_Text = open( path+'.face').read().split('\n')[:-1]
    for line in Faces_Text:
        line = line.split()
        face_vert = map(int , map(float,line) )
        factory.insert_element(face_vert)

    grid = factory.finalize()
    export_file = os.path.join(path + output_format )
    print('{0} File Ready'.format(output_format))
    bempp.api.export(grid=grid, file_name=export_file) 
    return None

def surf_to_mesh(name , suffix = '.pdb.output.surf.off' , output_format = '_surf.msh'):
    '''
    This rutine makes a .msh or .vtk file from .surf.off file obtained from GAMer
    
    name  : Molecule ID
    output_format : _surf.msh for msh format (GMSH) or _surf.vtk format (ParaView for example). VTK 
                    format allow us to see higher-order(>1) solutions. 
    '''
    path = os.path.join('Molecule',name,name)
    off_File = open(path + suffix).read().split('\n')

    vert_text = open(path+'_surf.vert','w+')
    c=0
    factory = bempp.api.grid.GridFactory()
    for line in off_File[2:]:
        l = map(float,line.split())
        if len(l)!=3: break
        c+=1
        vert_text.write(line+' \n')
        factory.insert_vertex(l)
    vert_text.close()

    face_text = open(path+'_surf.face','w+')
    for line in off_File[c+2:-1]:
        l_aux = line[2:]
        l = map(int,l_aux.split())
        face_text.write(l_aux+' \n')
        factory.insert_element(l)
    face_text.close()
    grid=factory.finalize()
    export_file = os.path.join(path + output_format )
    print('{0} File Ready'.format(output_format))
    bempp.api.export(grid=grid, file_name=export_file) 
    return None

In [4]:
mesher('born', suffix='.pdb.output.all.m')

ValueError: 'vert=[ ' is not in list

In [ ]:
'.pdb.output.all.m'

In [71]:
grid_factory('1CID', suffix='', output_format='.msh')

.msh File Ready


In [76]:
surf_to_mesh('1CID' , suffix = '.pdb.output.surf.off' , output_format = '_surf.msh')

_surf.msh File Ready
